Para executar os notebooks a seguir, se ainda não fez isso, você precisa implantar um modelo que use `text-embedding-ada-002` como modelo base e definir o nome da implantação dentro do arquivo .env como `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Em seguida, vamos carregar o Índice de Embeddings em um Dataframe do Pandas. O Índice de Embeddings está armazenado em um arquivo JSON chamado `embedding_index_3m.json`. O Índice de Embeddings contém os Embeddings para cada uma das transcrições do YouTube até o final de outubro de 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Em seguida, vamos criar uma função chamada `get_videos` que irá buscar no Índice de Embeddings pela consulta. A função retornará os 5 vídeos principais que são mais semelhantes à consulta. A função funciona da seguinte forma:

1. Primeiro, uma cópia do Índice de Embeddings é criada.
2. Em seguida, o Embedding para a consulta é calculado usando a API de Embeddings da OpenAI.
3. Depois, uma nova coluna é criada no Índice de Embeddings chamada `similarity`. A coluna `similarity` contém a similaridade do cosseno entre o Embedding da consulta e o Embedding de cada segmento de vídeo.
4. Em seguida, o Índice de Embeddings é filtrado pela coluna `similarity`. O Índice de Embeddings é filtrado para incluir apenas vídeos que tenham uma similaridade do cosseno maior ou igual a 0,75.
5. Finalmente, o Índice de Embeddings é ordenado pela coluna `similarity` e os 5 principais vídeos são retornados.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Esta função é muito simples, ela apenas imprime os resultados da consulta de pesquisa.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Primeiro, o Índice de Embeddings é carregado em um Dataframe do Pandas.
2. Em seguida, o usuário é solicitado a inserir uma consulta.
3. Então, a função `get_videos` é chamada para buscar no Índice de Embeddings pela consulta.
4. Finalmente, a função `display_results` é chamada para exibir os resultados para o usuário.
5. O usuário é então solicitado a inserir outra consulta. Esse processo continua até que o usuário digite `exit`.

![](../../../../translated_images/br/notebook-search.1e320b9c7fcbb0bc.png)

Você será solicitado a inserir uma consulta. Digite uma consulta e pressione enter. O aplicativo retornará uma lista de vídeos que são relevantes para a consulta. O aplicativo também retornará um link para o local no vídeo onde a resposta à pergunta está localizada.

Aqui estão algumas consultas para experimentar:

- O que é Azure Machine Learning?
- Como funcionam as redes neurais convolucionais?
- O que é uma rede neural?
- Posso usar Jupyter Notebooks com Azure Machine Learning?
- O que é ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
